---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The **first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket**. This is where predictive modeling comes in. For this assignment, **your task is to predict whether a given blight ticket will be paid on time**.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [211]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from datetime import datetime

In [6]:
train_data_path = '../data/train.csv'
train_df = pd.read_csv(train_data_path, encoding = 'ISO-8859-1')
train_df.head()

/Users/caboe/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [333]:
test_data_path = '../data/test.csv'
test_df = pd.read_csv(test_data_path, encoding = 'ISO-8859-1')
test_df.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 34 columns):
ticket_id                     250306 non-null int64
agency_name                   250306 non-null object
inspector_name                250306 non-null object
violator_name                 250272 non-null object
violation_street_number       250306 non-null float64
violation_street_name         250306 non-null object
violation_zip_code            0 non-null float64
mailing_address_str_number    246704 non-null float64
mailing_address_str_name      250302 non-null object
city                          250306 non-null object
state                         250213 non-null object
zip_code                      250305 non-null object
non_us_str_code               3 non-null object
country                       250306 non-null object
ticket_issued_date            250306 non-null object
hearing_date                  237815 non-null object
violation_code                250306 non-null obj

In [12]:
cols = ['ticket_id',
        'agency_name', 
        'violation_zip_code', 
        'zip_code', 'country', 
        'ticket_issued_date', 
        'violation_code',
        'fine_amount',
        'collection_status',
        'grafitti_status',
        'inspector_name',
        'compliance_detail',
        'compliance']

In [105]:
df1 = train_df[cols]
#df1.head()

In [116]:
good_rows = df1.compliance.notnull()
df_clean = df1[good_rows]

In [135]:
unique_fine_amounts = sorted(df_clean.fine_amount.unique())
len(unique_fine_amounts)

40

In [148]:
total_compliant = df_clean.compliance.value_counts()[1]

In [150]:
compliance_by_fine_amount = {}
for g,f in df_clean.groupby('fine_amount'):
    yes = sum(f.compliance)
    no = f.shape[0] - sum(f.compliance)
    rate = float(yes)/ f.shape[0]
    overall_rate = yes / total_compliant
    compliance_by_fine_amount[g] = rate,overall_rate
    print('Fine Amount {}'.format(g))
    print('Count {}'.format(f.shape[0]))
    print('Compliant : Yes {}, No {}'.format(yes,no))
    print('Compliance Rate {}'.format(rate))
    print('Overall Compliant Contribution {}'.format(overall_rate))
    print()

Fine Amount 0.0
Count 195
Compliant : Yes 195.0, No 0.0
Compliance Rate 1.0
Overall Compliant Contribution 0.016814693455203932

Fine Amount 1.0
Count 1
Compliant : Yes 1.0, No 0.0
Compliance Rate 1.0
Overall Compliant Contribution 8.622919720617402e-05

Fine Amount 20.0
Count 1
Compliant : Yes 1.0, No 0.0
Compliance Rate 1.0
Overall Compliant Contribution 8.622919720617402e-05

Fine Amount 25.0
Count 1378
Compliant : Yes 107.0, No 1271.0
Compliance Rate 0.07764876632801161
Overall Compliant Contribution 0.00922652410106062

Fine Amount 50.0
Count 20415
Compliant : Yes 1850.0, No 18565.0
Compliance Rate 0.09061964241978937
Overall Compliant Contribution 0.15952401483142192

Fine Amount 95.0
Count 2
Compliant : Yes 0.0, No 2.0
Compliance Rate 0.0
Overall Compliant Contribution 0.0

Fine Amount 100.0
Count 15488
Compliant : Yes 1814.0, No 13674.0
Compliance Rate 0.11712293388429752
Overall Compliant Contribution 0.15641976373199964

Fine Amount 125.0
Count 793
Compliant : Yes 40.0, No 75

In [163]:
total = 0
for k,v in compliance_by_fine_amount.items():
    if k <= 300:
        total += v[1]
total

0.93696645684228685

In [165]:
for k,v in compliance_by_fine_amount.items():
    print(k,':', v[0])

0.0 : 1.0
1.0 : 1.0
20.0 : 1.0
25.0 : 0.07764876632801161
50.0 : 0.09061964241978937
95.0 : 0.0
100.0 : 0.11712293388429752
125.0 : 0.05044136191677175
150.0 : 0.5
160.0 : 0.0
170.0 : 0.0
200.0 : 0.1
220.0 : 0.25
250.0 : 0.06310053227032882
270.0 : 1.0
300.0 : 0.028397027600849258
350.0 : 0.046875
400.0 : 0.05128205128205128
450.0 : 0.0
500.0 : 0.054061867591789534
600.0 : 0.0
655.0 : 0.0
677.0 : 0.0
750.0 : 0.0611353711790393
970.0 : 1.0
1000.0 : 0.03746223564954683
1030.0 : 1.0
1200.0 : 0.0
1250.0 : 0.0
1500.0 : 0.056818181818181816
1750.0 : 1.0
2000.0 : 0.125
2500.0 : 0.02977346278317152
2695.0 : 1.0
3000.0 : 0.0
3500.0 : 0.020730759264058048
5000.0 : 0.0
7000.0 : 0.0
8000.0 : 0.0
10000.0 : 0.005128205128205128


In [164]:
totals = []
total = 0
for k,v in compliance_by_fine_amount.items():
    total += v[1]
    totals.append(total)  
for amt, total in zip(unique_fine_amounts, totals):
    print(amt,':',total)

0.0 : 0.0168146934552
1.0 : 0.0169009226524
20.0 : 0.0169871518496
25.0 : 0.0262136759507
50.0 : 0.185737690782
95.0 : 0.185737690782
100.0 : 0.342157454514
125.0 : 0.345606622402
150.0 : 0.3456928516
160.0 : 0.3456928516
170.0 : 0.3456928516
200.0 : 0.455290161249
220.0 : 0.455376390446
250.0 : 0.927653703544
270.0 : 0.927739932741
300.0 : 0.936966456842
350.0 : 0.937483832026
400.0 : 0.93765629042
450.0 : 0.93765629042
500.0 : 0.969906010175
600.0 : 0.969906010175
655.0 : 0.969906010175
677.0 : 0.969906010175
750.0 : 0.971113218936
970.0 : 0.971199448133
1000.0 : 0.987238078813
1030.0 : 0.987324308011
1200.0 : 0.987324308011
1250.0 : 0.987324308011
1500.0 : 0.988617745969
1750.0 : 0.988790204363
2000.0 : 0.988962662758
2500.0 : 0.992929205829
2695.0 : 0.993015435026
3000.0 : 0.993015435026
3500.0 : 0.999913770803
5000.0 : 0.999913770803
7000.0 : 0.999913770803
8000.0 : 0.999913770803
10000.0 : 1.0


In [173]:
def less_than_500(x):
    if x <= 500:
        return True
    else:
        return False

In [174]:
df_clean['Less_Than_501'] = df_clean.fine_amount.apply(less_than_500)

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [175]:
df_clean.head()

,ticket_id,agency_name,violation_zip_code,zip_code,country,ticket_issued_date,violation_code,fine_amount,collection_status,grafitti_status,inspector_name,compliance_detail,compliance,Less_Than_501
0,22056,"Buildings, Safety Engineering & Env Department",NaN,60606,USA,2004-03-16 11:40:00,9-1-36(a),250.0,NaN,NaN,"Sims, Martinzie",non-compliant by no payment,0.0,True
1,27586,"Buildings, Safety Engineering & Env Department",NaN,48208,USA,2004-04-23 12:30:00,61-63.0600,750.0,NaN,NaN,"Williams, Darrin",compliant by late payment within 1 month,1.0,False
5,22046,"Buildings, Safety Engineering & Env Department",NaN,908041512,USA,2004-05-01 11:50:00,9-1-36(a),250.0,NaN,NaN,"Sims, Martinzie",non-compliant by no payment,0.0,True
6,18738,"Buildings, Safety Engineering & Env Department",NaN,48038,USA,2004-06-14 14:15:00,61-63.0500,750.0,NaN,NaN,"Williams, Darrin",non-compliant by no payment,0.0,False
7,18735,"Buildings, Safety Engineering & Env Department",NaN,48211,USA,2004-06-16 12:30:00,61-63.0100,100.0,NaN,NaN,"Williams, Darrin",non-compliant by no payment,0.0,True


In [127]:
df_clean[df_clean.fine_amount == 0.0]

,ticket_id,agency_name,violation_zip_code,zip_code,country,ticket_issued_date,violation_code,fine_amount,collection_status,grafitti_status,inspector_name,compliance_detail,compliance
185496,217648,Department of Public Works,NaN,48088,USA,2009-05-06 14:15:00,22-2-45,0.0,NaN,NaN,"Hayes, Billy J",compliant by no fine,1.0
195826,227911,Detroit Police Department,NaN,48210,USA,2009-08-22 16:15:00,22-2-43,0.0,NaN,NaN,"Sims, Jevon",compliant by no fine,1.0
197234,229915,Department of Public Works,NaN,48234,USA,2009-09-02 10:05:00,9-1-104,0.0,NaN,NaN,"Tidwell, Rhonda",compliant by no fine,1.0
197378,229961,Department of Public Works,NaN,48224,USA,2009-09-03 13:35:00,9-1-104,0.0,NaN,NaN,"Havard, Jacqueline",compliant by no fine,1.0
197687,230966,Department of Public Works,NaN,48234,USA,2009-09-08 14:45:00,22-2-88,0.0,NaN,NaN,"Hayes, Billy J",compliant by no fine,1.0
197999,230641,Department of Public Works,NaN,48205,USA,2009-09-10 13:55:00,9-1-104,0.0,NaN,NaN,"Havard, Jacqueline",compliant by no fine,1.0
198756,230968,Department of Public Works,NaN,48234,USA,2009-09-17 14:45:00,9-1-110(a),0.0,NaN,NaN,"Hayes, Billy J",compliant by no fine,1.0
198765,231611,Department of Public Works,NaN,48216,USA,2009-09-17 14:00:00,22-2-88,0.0,NaN,NaN,"Zizi, Josue",compliant by no fine,1.0
198827,231582,Department of Public Works,NaN,48216,USA,2009-09-17 14:00:00,9-1-104,0.0,NaN,NaN,"Zizi, Josue",compliant by no fine,1.0
199101,231874,Department of Public Works,NaN,48223,USA,2009-09-21 09:20:00,9-1-105,0.0,NaN,NaN,"McClain, Melvin",compliant by no fine,1.0


In [169]:
unique_violation_codes = df_clean.violation_code.unique()

In [172]:
type(df_clean.ticket_issued_date[0]

str

In [187]:
def datetime_split_month(cell):
    day_time = cell.split(" ")
    month = day_time[0]
    return month

In [188]:
def datetime_split_time(cell):
    day_time = cell.split(" ")
    time = day_time[1]
    return time

In [189]:
df_clean['ticket_issued_month'] = df_clean.ticket_issued_date.apply(datetime_split_month)

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [190]:
df_clean['ticket_issued_time'] = df_clean.ticket_issued_date.apply(datetime_split_time)

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [192]:
df_clean.drop('Ticket_Issued_Test',axis =1, inplace=True)

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [225]:
# convert the date to month only
df_clean['month'] = pd.to_datetime(df_clean['ticket_issued_month']).dt.month

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [320]:
# create day of the week field
df_clean['day'] = pd.to_datetime(df_clean['ticket_issued_date']).dt.dayofweek

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [321]:
df_clean.head()

,ticket_id,agency_name,violation_zip_code,zip_code,country,ticket_issued_date,violation_code,fine_amount,inspector_name,compliance,Less_Than_501,ticket_issued_month,ticket_issued_time,month,time,day
0,22056,"Buildings, Safety Engineering & Env Department",NaN,60606,USA,2004-03-16 11:40:00,9-1-36(a),250.0,"Sims, Martinzie",0.0,True,2004-03-16,11:40:00,3,11,1
1,27586,"Buildings, Safety Engineering & Env Department",NaN,48208,USA,2004-04-23 12:30:00,61-63.0600,750.0,"Williams, Darrin",1.0,False,2004-04-23,12:30:00,4,12,4
5,22046,"Buildings, Safety Engineering & Env Department",NaN,908041512,USA,2004-05-01 11:50:00,9-1-36(a),250.0,"Sims, Martinzie",0.0,True,2004-05-01,11:50:00,5,11,5
6,18738,"Buildings, Safety Engineering & Env Department",NaN,48038,USA,2004-06-14 14:15:00,61-63.0500,750.0,"Williams, Darrin",0.0,False,2004-06-14,14:15:00,6,14,0
7,18735,"Buildings, Safety Engineering & Env Department",NaN,48211,USA,2004-06-16 12:30:00,61-63.0100,100.0,"Williams, Darrin",0.0,True,2004-06-16,12:30:00,6,12,2


In [229]:
# drop columns that won't be used in the model since 
#they are not known at the time of ticket issue
df_clean.drop(['grafitti_status', 'compliance_detail', 'collection_status'], axis = 1, inplace=True)

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [233]:
#convert the full time stamp hourly buckets
df_clean['time'] = pd.to_datetime(df_clean['ticket_issued_time']).dt.hour

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [334]:
# Add the city feature
df_clean['city'] = train_df['city']

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [339]:
df_clean.city.value_counts()

DETROIT                87426
SOUTHFIELD              8790
Detroit                 6374
DEARBORN                2408
detroit                 2197
FARMINGTON HILLS        1468
OAK PARK                1436
WARREN                  1235
W. BLOOMFIELD           1033
DET                     1010
REDFORD                  951
TROY                     888
LIVONIA                  784
WEST BLOOMFIELD          724
Southfield               687
CANTON                   642
FARMINGTON               591
BLOOMFIELD HILLS         587
CHICAGO                  564
DET.                     546
SAN DIEGO                538
EASTPOINTE               537
ROSEVILLE                502
GROSSE POINTE            483
GILBERT                  480
LATHRUP VILLAGE          471
LANSING                  438
FERNDALE                 432
WESTLAND                 427
ST. CLAIR SHORES         418
                       ...  
DES MOINE                  1
BROWNSTOWN TWP.            1
WESDT BLOOMFIELD HI        1
LINVILLE      

In [242]:
def compliance_impact(group_name):
    compliance_by_ = {}
    for g,f in df_clean.groupby(group_name):
        yes = sum(f.compliance)
        no = f.shape[0] - sum(f.compliance)
        rate = float(yes)/ f.shape[0]
        overall_rate = yes / total_compliant
        compliance_by_[g] = rate#,overall_rate
#        print('Fine Amount {}'.format(g))
#        print('Count {}'.format(f.shape[0]))
#        print('Compliant : Yes {}, No {}'.format(yes,no))
#        print('Compliance Rate {}'z.format(rate))
#        print('Overall Compliant Contribution {}'.format(overall_rate))
#        print()
    return compliance_by_

In [243]:
compliance_by_month = compliance_impact('month')
compliance_by_month

{1: 0.08903383611903791,
 2: 0.09117541979278314,
 3: 0.09519408502772643,
 4: 0.07776898734177215,
 5: 0.0771782397169394,
 6: 0.06726802321515978,
 7: 0.06212574850299401,
 8: 0.05587152893944463,
 9: 0.0582209213540341,
 10: 0.05747349366433928,
 11: 0.06481930045538223,
 12: 0.08056407112201104}

In [245]:
compliance_by_hour = compliance_impact('time')
compliance_by_hour

{0: 0.048484848484848485,
 1: 0.075,
 2: 0.08633093525179857,
 3: 0.05357142857142857,
 4: 0.14285714285714285,
 5: 0.0,
 6: 0.16666666666666666,
 7: 0.08450704225352113,
 8: 0.060595938001068946,
 9: 0.05947611710323575,
 10: 0.07939021975846366,
 11: 0.07662011088556973,
 12: 0.08386485088777797,
 13: 0.07445974694882992,
 14: 0.08069855039379066,
 15: 0.0917813934084272,
 16: 0.11891891891891893,
 17: 0.08641975308641975,
 18: 0.144,
 19: 0.13559322033898305,
 20: 0.11363636363636363,
 21: 0.13414634146341464,
 22: 0.09166666666666666,
 23: 0.12195121951219512}

In [323]:
compliance_by_dayofweek = compliance_impact('day')
compliance_by_dayofweek

{0: 0.06840899625156185,
 1: 0.07332826747720365,
 2: 0.0731862292051756,
 3: 0.07351771400599645,
 4: 0.07227806337757998,
 5: 0.12,
 6: 0.12539184952978055}

In [324]:
def weekday_end(x):
    if x < 5:
        return 0
    else:
        return 1

In [325]:
df_clean['weekday'] = df_clean.day.apply(weekday_end)

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [326]:
compliance_by_weekend = compliance_impact('weekday')
compliance_by_weekend

{0: 0.0722729444905556, 1: 0.12203791469194313}

In [246]:
df_clean['agency_name'].unique()

array(['Buildings, Safety Engineering & Env Department',
       'Health Department', 'Department of Public Works',
       'Detroit Police Department', 'Neighborhood City Halls'], dtype=object)

In [340]:
compliance_by_city = compliance_impact('city')
compliance_by_city

{'0': 0.0,
 '1129': 0.0,
 '14943': 0.0,
 '19007': 0.0,
 '283 SHAKER HTS.': 0.0,
 '380133': 0.0,
 '4303': 0.0,
 '48037': 0.0,
 '48146': 0.0,
 '48203': 0.0,
 '48204': 0.0,
 '48206': 0.0,
 '48221': 0.0,
 '48223': 0.0,
 '48224': 0.0,
 '48227': 0.0,
 '48235': 0.0,
 '7136': 0.0,
 '77081': 0.0,
 ';INCOLN PARK': 1.0,
 'ABURN HILLS': 0.0,
 'ADA': 0.0,
 'ADAMSVILLE': 0.0,
 'ADDISON': 0.0,
 'ADDISON  TWP': 0.0,
 'ADRAIN': 0.0,
 'ADRIAN': 0.3333333333333333,
 'AGOURA HILL': 0.0,
 'AGOURA HILLS': 0.5,
 'AGUSTA': 0.0,
 'AKRON': 0.0,
 'ALAMEDA': 0.0,
 'ALAMOGORDO': 0.0,
 'ALANTA': 0.0,
 'ALBANY': 0.0,
 'ALBEQUERQUE': 0.0,
 'ALBERTA': 0.0,
 'ALBION': 0.0,
 'ALBUERGUE': 0.0,
 'ALBUGERQUE': 0.0,
 'ALBUQQUERQUE': 0.0,
 'ALBUQUERQUE': 0.0,
 'ALCONAC': 0.0,
 'ALDERGROVE': 0.0,
 'ALEXANDER': 0.0,
 'ALEXANDRA': 0.0,
 'ALEXANDRIA': 0.0,
 'ALEXANDRINE': 0.0,
 'ALFORTEVILLE, FRANCE': 0.0,
 'ALGONAC': 0.12121212121212122,
 'ALHAMBRA': 0.0,
 'ALISO': 0.0,
 'ALISO VIETO': 0.0,
 'ALLEN': 0.0,
 'ALLEN  PARK': 0.5,
 

In [350]:
compliant_city = []
for k,v in compliance_by_city.items():
    if v >= .15:
        compliant_city.append(k)
len(compliant_city)

548

In [344]:
compliance_by_zip = compliance_impact('zip_code')
compliance_by_zip

{0: 0.0,
 2: 0.0,
 48: 0.0,
 78: 0.0,
 127: 1.0,
 179: 0.0,
 237: 0.0,
 311: 0.0,
 342: 0.0,
 427: 0.0,
 482: 0.3333333333333333,
 677: 0.0,
 782: 0.0,
 831: 0.25,
 1041: 0.0,
 1119: 0.0,
 1401: 0.0,
 1603: 0.0,
 1606: 0.0,
 1620: 0.0,
 1817: 0.0,
 2029: 0.0,
 2067: 0.0,
 2116: 0.0,
 2210: 0.0,
 2222: 0.0,
 2365: 0.0,
 2368: 0.0,
 2448: 0.0,
 2494: 1.0,
 2510: 0.0,
 2812: 0.0,
 2895: 0.6,
 2903: 0.0,
 3129: 0.0,
 3236: 0.0,
 3522: 0.0,
 3661: 0.0,
 3973: 0.0,
 4066: 0.0,
 4226: 1.0,
 4304: 0.0,
 4390: 1.0,
 4804: 0.0,
 4821: 0.0,
 4823: 0.3333333333333333,
 4824: 0.0,
 4825: 0.0,
 4826: 0.0,
 4827: 1.0,
 4828: 0.0,
 4830: 0.0,
 4834: 0.0,
 4838: 1.0,
 4937: 0.0,
 4993: 0.0,
 5024: 0.0,
 5032: 0.0,
 6045: 1.0,
 6141: 0.0,
 6484: 0.0,
 6489: 0.0,
 6492: 0.0,
 6506: 0.0,
 6610: 0.0,
 6670: 0.0,
 6755: 0.0,
 6770: 0.0,
 6775: 0.0,
 6831: 0.0,
 6902: 0.0,
 7010: 0.0,
 7019: 1.0,
 7039: 0.0,
 7041: 0.0,
 7052: 0.0,
 7068: 0.0,
 7079: 0.0,
 7087: 0.0,
 7110: 1.0,
 7302: 0.0,
 7304: 0.5,
 7306

In [352]:
#compliant_zip = []
for k,v in compliance_by_zip.items():
    if k == None
#len(non_compliant_zip)

3543

In [354]:
len(df_clean.zip_code.unique())

4623

In [371]:
# impute 0 as zipcode on missing value
df_clean.zip_code[df_clean.zip_code.isnull()] = 0

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/caboe/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:4702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/caboe/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, sel

In [ ]:
compl

In [370]:
compliance_by_zip[]

KeyError: None

In [375]:
def bucketize_zipcodes(x):
    rate = compliance_by_zip[x]
    if rate == 0.:
        return 0
    elif ((rate > 0.) & (rate <= 0.08)):
        return 1
    elif ((rate > 0.075) & (rate <= .2)):
        return 2
    elif ((rate > .2) & (rate <= 0.5)):
        return 3
    else:
        return 4

In [376]:
df_clean['zip_buckets'] = df_clean.zip_code.apply(bucketize_zipcodes)

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [378]:
df_clean['zip_buckets'][:10]

0     2
1     1
5     0
6     2
7     2
8     2
9     1
12    1
13    1
14    1
Name: zip_buckets, dtype: int64

In [381]:
df_clean.columns

Index(['ticket_id', 'agency_name', 'violation_zip_code', 'zip_code', 'country',
       'ticket_issued_date', 'violation_code', 'fine_amount', 'inspector_name',
       'compliance', 'Less_Than_501', 'ticket_issued_month',
       'ticket_issued_time', 'month', 'time', 'day', 'weekday', 'city',
       'zip_buckets'],
      dtype='object')

In [382]:
df_clean

,ticket_id,agency_name,violation_zip_code,zip_code,country,ticket_issued_date,violation_code,fine_amount,inspector_name,compliance,Less_Than_501,ticket_issued_month,ticket_issued_time,month,time,day,weekday,city,zip_buckets
0,22056,"Buildings, Safety Engineering & Env Department",NaN,60606,USA,2004-03-16 11:40:00,9-1-36(a),250.0,"Sims, Martinzie",0.0,True,2004-03-16,11:40:00,3,11,1,0,CHICAGO,2
1,27586,"Buildings, Safety Engineering & Env Department",NaN,48208,USA,2004-04-23 12:30:00,61-63.0600,750.0,"Williams, Darrin",1.0,False,2004-04-23,12:30:00,4,12,4,0,Detroit,1
5,22046,"Buildings, Safety Engineering & Env Department",NaN,908041512,USA,2004-05-01 11:50:00,9-1-36(a),250.0,"Sims, Martinzie",0.0,True,2004-05-01,11:50:00,5,11,5,1,LOG BEACH,0
6,18738,"Buildings, Safety Engineering & Env Department",NaN,48038,USA,2004-06-14 14:15:00,61-63.0500,750.0,"Williams, Darrin",0.0,False,2004-06-14,14:15:00,6,14,0,0,Clinton,2
7,18735,"Buildings, Safety Engineering & Env Department",NaN,48211,USA,2004-06-16 12:30:00,61-63.0100,100.0,"Williams, Darrin",0.0,True,2004-06-16,12:30:00,6,12,2,0,Detroit,2
8,18733,"Buildings, Safety Engineering & Env Department",NaN,48211,USA,2004-06-16 12:25:00,61-63.0100,100.0,"Williams, Darrin",0.0,True,2004-06-16,12:25:00,6,12,2,0,Detroit,2
9,28204,"Buildings, Safety Engineering & Env Department",NaN,48205,USA,2004-07-12 13:30:00,61-63.0600,750.0,"Williams, Darrin",0.0,False,2004-07-12,13:30:00,7,13,0,0,Detroit,1
12,18743,"Buildings, Safety Engineering & Env Department",NaN,48213,USA,2004-07-28 12:15:00,61-63.0600,750.0,"Williams, Darrin",0.0,False,2004-07-28,12:15:00,7,12,2,0,Detroit,1
13,18741,"Buildings, Safety Engineering & Env Department",NaN,48235,USA,2004-08-05 15:00:00,61-63.0600,750.0,"Williams, Darrin",0.0,False,2004-08-05,15:00:00,8,15,3,0,Detroit,1
14,18978,"Buildings, Safety Engineering & Env Department",NaN,48213,USA,2004-08-23 14:15:00,61-63.0600,750.0,"Williams, Darrin",0.0,False,2004-08-23,14:15:00,8,14,0,0,Detroit,1


In [383]:
df_clean_dmy = pd.get_dummies(df_clean['agency_name'])

In [387]:
df_clean[df_clean_dmy.columns] = df_clean_dmy

/Users/caboe/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [389]:
df_clean.head()

,ticket_id,agency_name,violation_zip_code,zip_code,country,ticket_issued_date,violation_code,fine_amount,inspector_name,compliance,...,time,day,weekday,city,zip_buckets,"Buildings, Safety Engineering & Env Department",Department of Public Works,Detroit Police Department,Health Department,Neighborhood City Halls
0,22056,"Buildings, Safety Engineering & Env Department",NaN,60606,USA,2004-03-16 11:40:00,9-1-36(a),250.0,"Sims, Martinzie",0.0,...,11,1,0,CHICAGO,2,1,0,0,0,0
1,27586,"Buildings, Safety Engineering & Env Department",NaN,48208,USA,2004-04-23 12:30:00,61-63.0600,750.0,"Williams, Darrin",1.0,...,12,4,0,Detroit,1,1,0,0,0,0
5,22046,"Buildings, Safety Engineering & Env Department",NaN,908041512,USA,2004-05-01 11:50:00,9-1-36(a),250.0,"Sims, Martinzie",0.0,...,11,5,1,LOG BEACH,0,1,0,0,0,0
6,18738,"Buildings, Safety Engineering & Env Department",NaN,48038,USA,2004-06-14 14:15:00,61-63.0500,750.0,"Williams, Darrin",0.0,...,14,0,0,Clinton,2,1,0,0,0,0
7,18735,"Buildings, Safety Engineering & Env Department",NaN,48211,USA,2004-06-16 12:30:00,61-63.0100,100.0,"Williams, Darrin",0.0,...,12,2,0,Detroit,2,1,0,0,0,0


In [248]:
df_clean['agency_name'].value_counts()

Buildings, Safety Engineering & Env Department    95863
Department of Public Works                        52445
Health Department                                  7107
Detroit Police Department                          4464
Neighborhood City Halls                               1
Name: agency_name, dtype: int64

In [247]:
compliance_by_agency = compliance_impact('agency_name')
compliance_by_agency

{'Buildings, Safety Engineering & Env Department': 0.06074293523048517,
 'Department of Public Works': 0.08996091143102297,
 'Detroit Police Department': 0.13172043010752688,
 'Health Department': 0.06585056986070072,
 'Neighborhood City Halls': 0.0}

In [256]:
print(df_clean['inspector_name'].value_counts().shape)
df_clean['inspector_name'].value_counts()

(159,)


Morris, John            11604
Samaan, Neil J           8720
O'Neal, Claude           8075
Steele, Jonathan         6962
Devaney, John            6837
Hayes, Billy J           6385
Sloane, Bennie J         5624
Sims, Martinzie          5526
Zizi, Josue              5060
Doetsch, James           4337
Danielson, Keith D       3880
Gailes, Orbie J          3451
Jones, Leah              3014
Davis, Darlene           2770
Legge, Gerald            2598
Havard, Jacqueline       2435
Sharpe, Anthony          2401
Johnson, Lois            2062
Harris, Rickey           1947
DeRamer, Andrew          1946
Moore, David             1825
Frazier, Willie          1805
Karwowski, Stephen       1786
ELLARD, EVERETT          1759
Watson, Jerry            1674
Addison, Michael         1667
Talbert, Reginald        1633
Matthews, Delos          1618
Williamson, Lillett      1539
Shimko, James            1522
                        ...  
Madrigal, Michael          11
O'Neil, Vincent T          10
Doonan, Ch

In [250]:
compliance_by_inspector = compliance_impact('inspector_name')
compliance_by_inspector

{'Addison, Michael': 0.07318536292741452,
 'Ahmad, Muna I': 0.06678383128295255,
 'Anderson, Trevis': 0.2,
 'Anding, Dwight': 0.0,
 'BENNETT, MARGARET': 0.5,
 'Baker, Kenneth': 0.06912442396313365,
 'Barela, Ralph': 0.07668944570994685,
 'Baumgardner, Robert P': 0.04956896551724138,
 'Baxendale, Daniel': 0.0,
 'Bell, Maydell': 0.06602373887240356,
 'Bennett, Margaret': 0.10591133004926108,
 'Berendt, Raymond': 0.0,
 'Bickers-Holmes, Amanda': 0.08043310131477185,
 'Black, Norma': 0.09210526315789473,
 'Borden, Kenyetta': 0.11194029850746269,
 'Brackett, Ross': 0.042704626334519574,
 'Brinkley, Kevin': 0.24,
 'Brooks, Eric': 0.05994550408719346,
 'Buchanan, Daryl': 0.08439897698209718,
 'Buckman, Keith': 0.07692307692307693,
 'Carroll, Frank': 0.0,
 'Carver, Gharian': 0.09121851599727705,
 'Cato, Valesta': 0.09822263797942002,
 'Clark, Marcel': 0.0,
 'Coleman, Lanetha': 1.0,
 'Coleman, Robert': 0.0,
 'Copty, Anton': 0.19895287958115182,
 'Cox, Timothy': 0.09090909090909091,
 'Crawford, K

In [252]:
df_clean['compliance'][df_clean.inspector_name == 'Crowder, Michael'].value_counts()

0.0    29
1.0     9
Name: compliance, dtype: int64

In [261]:
compliance_by_violation_code = compliance_impact('violation_code')
compliance_by_violation_code

{'19420901': 0.25,
 '19450901': 0.027777777777777776,
 '19830901': 0.0,
 '19840901': 0.5,
 '19850901': 0.0,
 '20130901': 0.038461538461538464,
 '20160901': 0.0,
 '20180901': 0.13333333333333333,
 '22-2-16': 0.0,
 '22-2-17': 0.07072135785007072,
 '22-2-17(a)': 0.11267605633802817,
 '22-2-18': 0.0,
 '22-2-20': 0.3333333333333333,
 '22-2-21(a)': 0.0,
 '22-2-21(b)': 0.12432432432432433,
 '22-2-22': 0.1003062787136294,
 '22-2-22(a)': 0.125,
 '22-2-23': 0.13793103448275862,
 '22-2-38': 0.0,
 '22-2-41': 0.02631578947368421,
 '22-2-42': 0.0,
 '22-2-43': 0.13242453748782862,
 '22-2-44': 0.12244897959183673,
 '22-2-45': 0.1319047619047619,
 '22-2-49': 0.2,
 '22-2-49(a)': 0.0,
 '22-2-49(b)': 0.0,
 '22-2-49(c)': 0.0,
 '22-2-49(d)': 0.2,
 '22-2-53': 0.0,
 '22-2-55': 0.14285714285714285,
 '22-2-56': 0.14666666666666667,
 '22-2-61': 0.24787535410764872,
 '22-2-83': 0.043227665706051875,
 '22-2-83(a)(b)(c)': 0.032388663967611336,
 '22-2-83(b)': 0.047619047619047616,
 '22-2-83(c)': 0.13432835820895522,

In [269]:
len(df_clean.violation_code.value_counts())

189

In [391]:
df_clean_dmy.columns

Index(['Buildings, Safety Engineering & Env Department',
       'Department of Public Works', 'Detroit Police Department',
       'Health Department', 'Neighborhood City Halls'],
      dtype='object')

In [390]:
df_clean.columns

Index(['ticket_id', 'agency_name', 'violation_zip_code', 'zip_code', 'country',
       'ticket_issued_date', 'violation_code', 'fine_amount', 'inspector_name',
       'compliance', 'Less_Than_501', 'ticket_issued_month',
       'ticket_issued_time', 'month', 'time', 'day', 'weekday', 'city',
       'zip_buckets', 'Buildings, Safety Engineering & Env Department',
       'Department of Public Works', 'Detroit Police Department',
       'Health Department', 'Neighborhood City Halls'],
      dtype='object')

In [292]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.linear_model import LinearRegression, LogisticRegression

In [392]:
def to_categorical(df, column):
    df[column] = df[column].astype('category').cat.codes
    df[column] = df[column].astype('category')

In [393]:
df_clean.head()

,ticket_id,agency_name,violation_zip_code,zip_code,country,ticket_issued_date,violation_code,fine_amount,inspector_name,compliance,...,time,day,weekday,city,zip_buckets,"Buildings, Safety Engineering & Env Department",Department of Public Works,Detroit Police Department,Health Department,Neighborhood City Halls
0,22056,"Buildings, Safety Engineering & Env Department",NaN,60606,USA,2004-03-16 11:40:00,9-1-36(a),250.0,"Sims, Martinzie",0.0,...,11,1,0,CHICAGO,2,1,0,0,0,0
1,27586,"Buildings, Safety Engineering & Env Department",NaN,48208,USA,2004-04-23 12:30:00,61-63.0600,750.0,"Williams, Darrin",1.0,...,12,4,0,Detroit,1,1,0,0,0,0
5,22046,"Buildings, Safety Engineering & Env Department",NaN,908041512,USA,2004-05-01 11:50:00,9-1-36(a),250.0,"Sims, Martinzie",0.0,...,11,5,1,LOG BEACH,0,1,0,0,0,0
6,18738,"Buildings, Safety Engineering & Env Department",NaN,48038,USA,2004-06-14 14:15:00,61-63.0500,750.0,"Williams, Darrin",0.0,...,14,0,0,Clinton,2,1,0,0,0,0
7,18735,"Buildings, Safety Engineering & Env Department",NaN,48211,USA,2004-06-16 12:30:00,61-63.0100,100.0,"Williams, Darrin",0.0,...,12,2,0,Detroit,2,1,0,0,0,0


In [402]:
print(df_clean.columns)

Index(['ticket_id', 'agency_name', 'violation_zip_code', 'zip_code', 'country',
       'ticket_issued_date', 'violation_code', 'fine_amount', 'inspector_name',
       'compliance', 'Less_Than_501', 'ticket_issued_month',
       'ticket_issued_time', 'month', 'time', 'day', 'weekday', 'city',
       'zip_buckets', 'Buildings, Safety Engineering & Env Department',
       'Department of Public Works', 'Detroit Police Department',
       'Health Department', 'Neighborhood City Halls'],
      dtype='object')


In [409]:
to_categorical(df_clean,'agency_name')

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [427]:
df_clean2 = df_clean[:]

In [ ]:
to_categorical(df_clean2, 'month')
to_categorical(df_clean2, 'time')
to_categorical(df_clean2, 'weekday')
to_categorical(df_clean2, 'zip_buckets')

In [410]:
# roc_auc = 0.69721405034745265
current_cols = ['fine_amount', 'month', 'time', 'weekday','zip_buckets']

In [418]:
# roc_auc = 0.70311063246207361
current_cols = ['fine_amount', 'month', 'time', 'weekday','zip_buckets', 'agency_name']

In [434]:
X = df_clean2[current_cols]
y = df_clean2['compliance']

In [435]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

In [437]:
m = GradientBoostingClassifier(random_state = 0)
#m = LogisticRegression(random_state = 0) 

m.fit(X_train, y_train)
print('Training Accuracy:', m.score(X_train, y_train))
print('Test Accuracy: ', m.score(X_test, y_test))

Training Accuracy: 0.930789758986
Test Accuracy:  0.930698023518


In [438]:
#compute AUC score for initial model
y_score_m = m.fit(X_train, y_train).decision_function(X_test)
fpr_m, tpr_m, _ = roc_curve(y_test, y_score_m)
roc_auc_m = auc(fpr_m, tpr_m)

In [439]:
roc_auc_m

0.70311063246207361

In [425]:
roc_auc_clf

0.62262366867887264

In [ ]:
test_path = ''

In [316]:
np.sort(y_score_m)

array([-4.6424486 , -4.54718902, -4.54718902, ..., -0.94841261,
       -0.77738294, -0.77738294])

In [426]:
current_cols


['fine_amount', 'month', 'time', 'weekday', 'zip_buckets', 'agency_name']

In [ ]:
test_df

In [ ]:
y_test = 

In [327]:
# baseline accuracy
#df2.compliance.value_counts(1)

0.0    0.927464
1.0    0.072536
Name: compliance, dtype: float64

In [328]:
# Check value_counts against DummyClassifier Score
from sklearn.dummy import DummyClassifier

# Negative class (0) is most frequent
dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
# Therefore the dummy 'most_frequent' classifier always predicts class 0
y_dummy_predictions = dummy_majority.predict(X_test)

y_dummy_predictions

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [331]:
from sklearn.dummy import DummyClassifier

# Negative class (0) is most frequent
dummy_majority = DummyClassifier(strategy='prior').fit(X_train, y_train)
# Therefore the dummy 'most_frequent' classifier always predicts class 0
y_dummy_predictions = dummy_majority.predict(X_test)

y_dummy_predictions

array([ 0.])

In [41]:
train_df['grafitti_status'][train_df.grafitti_status.notnull()]

250296    GRAFFITI TICKET
Name: grafitti_status, dtype: object

In [45]:
df2.drop('grafitti_status', inplace=True , axis = 1)

/Users/caboe/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [46]:
df2.columns

Index(['ticket_id', 'agency_name', 'violation_zip_code', 'zip_code', 'country',
       'ticket_issued_date', 'violation_code', 'fine_amount',
       'collection_status', 'inspector_name', 'compliance_detail',
       'compliance'],
      dtype='object')

In [26]:
from sklearn.metrics import confusion_matrix

In [27]:
confusion_matrix?

In [25]:
train_df.violator_name.value_counts()

INVESTMENT, ACORN                         809
INVESTMENT CO., ACORN                     425
BANK, WELLS FARGO                         328
MILLER, JOHN                              205
SHIFMAN, ALLEN                            192
NEW YORK, BANK OF                         184
COMMISSION, DETROIT HOUSING               178
STEHLIK, JERRY                            162
SNOW, GEORGE                              147
TREASURER, WAYNE COUNTY                   139
APARTMENTS, CARLTON                       139
KRAMER, KEITH                             137
FARWELL, WILLIAM B                        123
NATIONAL TRUST CO., DEUTSCHE BANK         122
INVESTMENTS, ACORN                        122
PARKVIEW LLC, NORMANDY                    122
BANK, DEUTSCHE                            119
ROSEN, NATHAN                             110
PRESTIGE, KMG                             101
LEONARD, JACK                              99
BROADWAY, LLC, 1509                        98
SYSTEMS, MANAGEMENT               

In [ ]:
for group, frame in train_df.groupby('violator_name'):

In [57]:
df1.compliance.notnull().value_counts()

True     159880
False     90426
Name: compliance, dtype: int64

In [68]:
train_df[['country', 'compliance']][train_df.country != 'USA']

,country,compliance
160652,Cana,0.0
177864,Aust,1.0
178520,Aust,NaN
211755,Cana,0.0
216567,Egyp,0.0
216568,Egyp,1.0
216927,Cana,0.0
217529,Cana,NaN
222230,Cana,0.0
226259,Germ,0.0


In [83]:
v_name = train_df.columns[3]

In [94]:
for idx,row in train_df.iterrows():
    if row[v_name] == 'INVESTMENT, ACORN':
        print(row['compliance'])

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
0.0
0.0
0.0
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
0.0
0.0
0.0
0.0
0.0
0.0
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
0.0
0.0
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
nan
nan
0.0
0.0
nan
nan
0.0
0.0
0.0
0.0
nan
0.0
nan
nan
0.0
0.0
0.0
0.0
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
0.0
0.0
nan
0.0
0.0
nan
0.0
0.0
0.0
nan
0.0
nan
nan
0.0
nan
0.0
0.0
nan
0.0
0.0
0.0
0.0
nan
nan
nan
0.0
nan
nan
0.0
nan
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
0.0
0.0
0.0
0.0
0.0
0.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
nan
nan
nan
nan
0.0
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
0.0
nan
0.0
nan
0.0
nan
nan
nan
0.0
nan
0.0
0.0
0.0
0.0
0.0
0.0
nan
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
nan
0.0
nan
nan
0.0
nan
nan
0.0
0.0
0.0
0.0
0.0
nan
0.0


In [97]:
for name in train_df.violator_name.value_counts()[:100]:
    if 'Acorn' in name:
        print(name)

TypeError: argument of type 'numpy.int64' is not iterable

In [101]:
values = train_df.violator_name.value_counts()[:100].index.tolist()
count = train_df.violator_name.value_counts()[:100]

In [103]:
values

['INVESTMENT, ACORN',
 'INVESTMENT CO., ACORN',
 'BANK, WELLS FARGO',
 'MILLER, JOHN',
 'SHIFMAN, ALLEN',
 'NEW YORK, BANK OF',
 'COMMISSION, DETROIT HOUSING',
 'STEHLIK, JERRY',
 'SNOW, GEORGE',
 'TREASURER, WAYNE COUNTY',
 'APARTMENTS, CARLTON',
 'KRAMER, KEITH',
 'FARWELL, WILLIAM B',
 'NATIONAL TRUST CO., DEUTSCHE BANK',
 'INVESTMENTS, ACORN',
 'PARKVIEW LLC, NORMANDY',
 'BANK, DEUTSCHE',
 'ROSEN, NATHAN',
 'PRESTIGE, KMG',
 'LEONARD, JACK',
 'BROADWAY, LLC, 1509',
 'SYSTEMS, MANAGEMENT',
 'Investment, Acorn',
 'ESTATES, EVERGREEN',
 'NATIONAL TRUST, DEUTSCHE BANK',
 'MANAGEMENT, FOREST CITY',
 'FARGO BANK, WELLS',
 'CHASE BANK, JP MORGAN',
 'MORTGAGE ASSOCIATION, FEDERAL NATIONAL',
 'INVESTMENT CO, ACORN',
 'SACHS, RALPH',
 'HOLDINGS LLC, PARAMOUNT LAND',
 'NATIONAL TRUST CO, DEUTSCHE BANK',
 'LDHA, BARCLAY',
 'ROSE, MAURICE',
 'FRIEDMAN, SINSILVER',
 'VILLAS, GENESIS',
 'VOID, VOID',
 'MANAGEMENT, JOY',
 'CONSTRUCTION, REX',
 'SYSTEMS INC, MANAGEMENT',
 'HOUSING COMMISSION, DETRO

In [317]:
df_clean.head()

,ticket_id,agency_name,violation_zip_code,zip_code,country,ticket_issued_date,violation_code,fine_amount,inspector_name,compliance,Less_Than_501,ticket_issued_month,ticket_issued_time,month,time
0,22056,"Buildings, Safety Engineering & Env Department",NaN,60606,USA,2004-03-16 11:40:00,9-1-36(a),250.0,"Sims, Martinzie",0.0,True,2004-03-16,11:40:00,3,11
1,27586,"Buildings, Safety Engineering & Env Department",NaN,48208,USA,2004-04-23 12:30:00,61-63.0600,750.0,"Williams, Darrin",1.0,False,2004-04-23,12:30:00,4,12
5,22046,"Buildings, Safety Engineering & Env Department",NaN,908041512,USA,2004-05-01 11:50:00,9-1-36(a),250.0,"Sims, Martinzie",0.0,True,2004-05-01,11:50:00,5,11
6,18738,"Buildings, Safety Engineering & Env Department",NaN,48038,USA,2004-06-14 14:15:00,61-63.0500,750.0,"Williams, Darrin",0.0,False,2004-06-14,14:15:00,6,14
7,18735,"Buildings, Safety Engineering & Env Department",NaN,48211,USA,2004-06-16 12:30:00,61-63.0100,100.0,"Williams, Darrin",0.0,True,2004-06-16,12:30:00,6,12


In [102]:
for name, occurences in zip(values, count):
    if 'Acorn' in name:
        print(name)

Investment, Acorn


In [ ]:
train_df[['fine_amount', 'compliance']][train_df['violator_name' == 'INVESTMENT, ACORN']]

In [ ]:



def blight_model():
    from sklearn.ensemble import RandomForestClassifier
    # Your code here
    
    return # Your answer here